In [1]:
%%capture
!pip install --upgrade -tensorflow_hub
# !pip install -U -huggingface_hub

import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"]="1"

import textattack
import transformers
import torch
import time
from datasets import Dataset
import sys
import hashlib
import numpy as np

from transformers import AutoTokenizer, AutoModelForSequenceClassification, BertForMaskedLM, pipeline
from textattack.attack_recipes import (
    TextBuggerLi2018, DeepWordBugGao2018, TextFoolerJin2019, BERTAttackLi2020
)
from textattack.constraints.semantics.sentence_encoders import UniversalSentenceEncoder
from textattack.models.wrappers import ModelWrapper

sys.path.append('../')
from eval_utils import *
sys.path.pop()

2023-08-05 20:04:07.119602: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-05 20:04:08.329061: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# set a seed, because reproducability is cool
np.random.seed(int(hashlib.sha256('Harrison Gietz'.encode('utf-8')).hexdigest(), 16) % 2**32)
torch.cuda.empty_cache()

device = input('enter a device name to run on: ')
dataset_val = input('Enter the number of samples to run on (100 or 776): ')
defense = input('Specify a defense type among "default", "logit", "maj_log", "one_hot": ')
cand_size = int(input('enter number of candidates (recommended 12 for quicker run, 50 otherwise): '))

imdb_tokenizer = AutoTokenizer.from_pretrained("textattack/bert-base-uncased-imdb")
imdb_model = AutoModelForSequenceClassification.from_pretrained("textattack/bert-base-uncased-imdb")
imdb_model.to(device)
imdb_pipeline = pipeline('sentiment-analysis', model=imdb_model, tokenizer=imdb_tokenizer)
imdb_pipeline.device = next(imdb_model.parameters()).device

imdb_model_directory = "../../../models/bert-uncased_maskedlm_imdb_july31_chk3"
finetuned_imdb_maskedlm = BertForMaskedLM.from_pretrained(imdb_model_directory)
finetuned_imdb_maskedlm.to(device)
imdb_fill_mask = pipeline("fill-mask", model=finetuned_imdb_maskedlm, tokenizer=imdb_tokenizer)
imdb_fill_mask.device = next(imdb_model.parameters()).device

num_voter = 11
mask_pct = 0.3    
    
attack = TextFoolerJin2019

if dataset_val == '100':
    loaded_imdb_100 = Dataset.load_from_disk('../data/filtered_imdb_clean_100')
    imdb_100 = textattack.datasets.Dataset(convert_to_tuples(loaded_imdb_100))
    dataset = imdb_100
    dataset_name = 'imdb100'
elif dataset_val =='776':
    loaded_imdb_776 = Dataset.load_from_disk('../data/filtered_imdb_clean_776')
    imdb_776 = textattack.datasets.Dataset(convert_to_tuples(loaded_imdb_776))
    dataset = imdb_776
    dataset_name = 'imdb776'
else:
    raise ValueError('Number of samples not supported')
    
if defense == "default":
    imdb_wrapper = textattack.models.wrappers.HuggingFaceModelWrapper(imdb_model, imdb_tokenizer)
elif defense == "logit":
    imdb_wrapper = MaskDemaskWrapper(imdb_model, imdb_tokenizer, imdb_fill_mask, num_voter, mask_pct, 'logit')
elif defense == 'maj_log':
    imdb_wrapper = MaskDemaskWrapper(imdb_model, imdb_tokenizer, imdb_fill_mask, num_voter, mask_pct, 'maj_log')
elif defense == "one_hot":
    imdb_wrapper = MaskDemaskWrapper(imdb_model, imdb_tokenizer, imdb_fill_mask, num_voter, mask_pct, 'maj_one_hot')
else:
    raise ValueError('Not a valid defense type.')
    
print(f'using num_voter = {num_voter} and mask_pct = {mask_pct} with dataset = {dataset_name}...')

# Parse the attack name
attack_name = parse_attack_name(attack)
attack = attack.build(imdb_wrapper)

# change candidate size
attack.transformation.max_candidates = cand_size
# adjust attack threshold to match Li et al. 2023 (0.7 theshold for imdb Universal sentences encoder):
attack.constraints[2] = UniversalSentenceEncoder(metric = 'angular', threshold = 0.7, 
                                                 window_size = 15, skip_text_shorter_than_window=True, 
                                                 compare_against_original=False)

# Set up arguments for the attack
attack_args = textattack.AttackArgs(
    num_examples=len(dataset),
    log_to_csv=f'{attack_name}_{dataset_name}_candsize{cand_size}_mp{mask_pct}_nv{num_voter}_{defense}_log.csv',
    checkpoint_interval=25, 
    checkpoint_dir="chkpts_2", 
    disable_stdout=True
)
# Perform the attack and save the results
attacker = textattack.Attacker(attack, dataset, attack_args)
attacker.attack_dataset()

print(f'The above are results for {attack_name}_{dataset_name}_candsize{cand_size}_mp{mask_pct}_nv{num_voter}_{defense}.')

enter a device name to run on: cuda:0
Enter the number of samples to run on (100 or 776): 776
Specify a defense type among "default", "logit", "maj_log", "one_hot": default
enter number of candidates (recommended 12 for quicker run, 50 otherwise): 12
using num_voter = 11 and mask_pct = 0.3 with dataset = imdb776...
Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  12
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
  

  0%|          | 0/776 [00:00<?, ?it/s]2023-08-05 20:09:23.025887: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2023-08-05 20:09:25.456827: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype string
	 [[{{node inputs}}]]
[Succeeded / Failed / Skipped / Total] 23 / 1 / 1 / 25:   3%|▎         | 25/776 [06:46<3:23:21, 16.25s/it]

[Succeeded / Failed / Skipped / Total] 44 / 4 / 2 / 50:   6%|▋         | 50/776 [13:15<3:12:36, 15.92s/it]

[Succeeded / Failed / Skipped / Total] 64 / 5 / 6 / 75:  10%|▉         | 75/776 [19:36<3:03:12, 15.68s/it]

[Succeeded / Failed / Skipped / Total] 84 / 8 / 8 / 100:  13%|█▎        | 100/776 [26:39<3:00:09, 15.99s/it]

...

[Succeeded / Failed / Skipped / Total] 635 / 38 / 52 / 725:  93%|█████████▎| 725/776 [2:42:36<11:26, 13.46s/it]

[Succeeded / Failed / Skipped / Total] 656 / 40 / 54 / 750:  97%|█████████▋| 750/776 [2:47:14<05:47, 13.38s/it]

[Succeeded / Failed / Skipped / Total] 678 / 40 / 57 / 775: 100%|█████████▉| 775/776 [2:53:29<00:13, 13.43s/it]

[Succeeded / Failed / Skipped / Total] 678 / 41 / 57 / 776: 100%|██████████| 776/776 [2:53:52<00:00, 13.44s/it]



+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 678    |
| Number of failed attacks:     | 41     |
| Number of skipped attacks:    | 57     |
| Original accuracy:            | 92.65% |
| Accuracy under attack:        | 5.28%  |
| Attack success rate:          | 94.3%  |
| Average perturbed word %:     | 13.46% |
| Average num. words per input: | 158.3  |
| Avg num queries:              | 334.41 |
+-------------------------------+--------+
The above are results for TextFoolerJin2019_imdb776_candsize12_mp0.3_nv11_default.
